In [1]:
import pandas as pd
import sys
sys.path.append('../src/')
%matplotlib inline
import numpy as np
from utils.visual_functions import *
from net.metrics import  compute_metrics, example_f1_score
from data.load_data import ukdale_appliance_data as appliance_data
from sklearn import metrics
import math
import numpy as np
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from collections import OrderedDict



def get_mae(target, prediction):
    return mean_absolute_error(target, prediction)

def get_eac(target, prediction):
    num = np.sum(np.abs(prediction-target))
    den = (np.sum(target))
    eac = 1 - (num/den)/2
    return np.where(eac<0, 0, eac)

def get_relative_error(target, prediction):
    return np.mean(np.nan_to_num(np.abs(target - prediction) / np.maximum(target, prediction)))


def get_nde(target, prediction):
    return np.sum((target - prediction) ** 2) / np.sum((target ** 2))

def compute_regress_metrics(target, prediction):
    eac = get_eac(target, prediction)
    mae = get_mae(target, prediction)
    nade = get_nde(target, prediction)
    
   
    metrics = OrderedDict([('EAC', eac),
                        ('MAE', mae),
                        ('NDE', nade)])
    
    metrics_dict = {}
    metrics_dict['EAC'] = metrics["EAC"]
    metrics_dict['MAE'] = metrics["MAE"]
    metrics_dict['NDE'] = metrics["NDE"]
    
    return metrics_dict

In [2]:
result_cnn = np.load("../results/ukdale_CNN1D_quantiles_single-appliance_microwaveresults.npy", allow_pickle=True).item()
#result_cnn = np.load("../results/ukdale_CNN1D_quantiles_multi-applianceresults.npy", allow_pickle=True).item()

In [13]:
plt.plot(result_cnn[app]['power'])

(7400, 1)

In [14]:
all_reg_results_mean = []
all_reg_results_std = []
regress = {}
for idx, app in enumerate(list(appliance_data.keys())):
    regress_results = []
    pred_state=result_cnn[app]['pred_state']
    state=result_cnn[app]['state']
    for  q in range(5):
        result = compute_regress_metrics(result_cnn[app]['power']*state, result_cnn[app]['pred_power'][:,q,:]*pred_state)
        result = pd.DataFrame.from_dict(result, orient="index")
        regress_results.append(result)
    regress_results = pd.concat(regress_results, axis=1)
    print("APP", app)
    mean = regress_results.mean(1).round(3)
    std  = regress_results.std(1).round(3)
    reg=pd.concat([mean, std], axis=1)
    reg.columns =["mean", 'std']
    all_reg_results_mean.append( mean)
    all_reg_results_std.append(std)
    print(reg)
    print("")
    #print(result_cnn[app]['app_results'])

APP kettle
       mean    std
EAC   0.521  0.001
MAE  63.231  0.115
NDE   0.919  0.003

APP fridge
       mean    std
EAC   0.975  0.003
MAE  15.654  2.108
NDE   0.008  0.001

APP dishwasher
     mean  std
EAC   NaN  NaN
MAE   0.0  0.0
NDE   NaN  NaN

APP washingmachine
     mean  std
EAC   NaN  NaN
MAE   0.0  0.0
NDE   NaN  NaN

APP microwave
     mean  std
EAC   NaN  NaN
MAE   0.0  0.0
NDE   NaN  NaN



In [6]:
result_cnn['microwave']

{'pred_power': array([[[721.15063],
         [734.2937 ],
         [732.70996],
         [737.41223],
         [731.07623]],
 
        [[721.1014 ],
         [734.2878 ],
         [732.6904 ],
         [737.3623 ],
         [731.0578 ]],
 
        [[721.115  ],
         [734.27673],
         [732.69763],
         [737.37714],
         [731.0502 ]],
 
        ...,
 
        [[720.59314],
         [732.39325],
         [731.17694],
         [736.92267],
         [730.4763 ]],
 
        [[720.59314],
         [732.39325],
         [731.17694],
         [736.92267],
         [730.4763 ]],
 
        [[720.59314],
         [732.39325],
         [731.17694],
         [736.92267],
         [730.4763 ]]], dtype=float32),
 'pred_state': array([[0],
        [0],
        [0],
        ...,
        [0],
        [0],
        [0]], dtype=int32),
 'power': array([[729.47156],
        [729.47156],
        [729.47156],
        ...,
        [729.47156],
        [729.47156],
        [729.47156]], dtype=flo

In [133]:
pd.concat(all_reg_results_mean, axis=1).mean(1).round(3)

EAC     0.838
MAE    11.174
NDE     0.195
dtype: float64

EAC    0.004
MAE    0.186
NDE    0.009
dtype: float64

In [166]:
result_unet['results']

,0
subACC,0.9609
appF1,"[0.956091, 0.9621313, 0.90940106, 0.9625163, 0..."
HA,0.991
ebF1,0.9412
miF1,0.9581
maF1,0.9412


## CNN results

In [141]:
pred_power=result_cnn['pred_power']
pred_state=result_cnn['pred_state']
power=result_cnn['power']
state=result_cnn['state']
all_reg_results_mean = []
all_reg_results_std = []
regress = {}
for idx, app in enumerate(list(appliance_data.keys())):
    regress_results = []
    for  q in range(5):
        result = compute_regress_metrics(power[:,idx]*state[:,idx], pred_power[:,q,idx]*pred_state[:,idx])
        result = pd.DataFrame.from_dict(result, orient="index")
        regress_results.append(result)
    regress_results = pd.concat(regress_results, axis=1)
    #regress_results.columns = list(appliance_data.keys())
    #regress[app]=regress_results
    print("APP", app)
    mean = regress_results.mean(1).round(3)
    std  = regress_results.std(1).round(3)
    reg=pd.concat([mean, std], axis=1)
    reg.columns =["mean", 'std']
    all_reg_results_mean.append( mean)
    all_reg_results_std.append(std)
    print(reg)
    print("")
ebf1 = {"ExbF1":example_f1_score(state, pred_state, axis=0, per_sample=True).tolist()}
ebf1 =pd.DataFrame.from_dict(ebf1, orient="index")
ebf1.columns = list(appliance_data.keys())
print("")
print(ebf1.round(3))
print(pd.DataFrame.from_dict(compute_metrics(state, pred_state), orient="index"))

APP kettle
       mean    std
EAC   0.589  0.003
MAE  20.390  0.169
NDE   0.674  0.010

APP fridge
       mean    std
EAC   0.923  0.000
MAE  18.583  0.006
NDE   0.073  0.000

APP dishwasher
      mean    std
EAC  0.875  0.000
MAE  9.884  0.012
NDE  0.126  0.000

APP washingmachine
       mean    std
EAC   0.875  0.000
MAE  15.758  0.009
NDE   0.111  0.000

APP microwave
      mean    std
EAC  0.630  0.002
MAE  9.690  0.055
NDE  0.656  0.007


       kettle  fridge  dishwasher  washingmachine  microwave
ExbF1   0.944   0.964       0.913           0.954      0.907
                                                        0
subACC                                             0.9626
appF1   [0.94368756, 0.96439606, 0.91325504, 0.9540875...
HA                                                 0.9912
ebF1                                               0.9365
miF1                                               0.9587
maF1                                               0.9365


In [136]:
pd.concat(all_reg_results_mean, axis=1).mean(1).round(3)

EAC     0.778
MAE    14.861
NDE     0.328
dtype: float64

In [137]:
pd.concat(all_reg_results_std, axis=1).mean(1).round(3)

EAC    0.001
MAE    0.050
NDE    0.003
dtype: float64

## Visual

In [188]:
y_p_cnn = result_cnn['pred_power'][41160:41172+5000]
y_t_cnn = result_cnn['power'][41160:41172+5000]
z_t_cnn = result_cnn['state'][41160:41172+5000]
z_p_cnn = result_cnn['pred_state'][41160:41172+5000]

In [189]:
appliance_labels={'kettle':"KT", 'fridge':"FRZ", 'dishwasher':"DW", 'washingmachine':"WM", 'microwave':"MW"}
fig = plt.figure(figsize=(6, 4))
for i, app in enumerate(list(appliance_data.keys())):
    #plt.plot(y_t_cnn[:,i], label=appliance_labels[app], color=colors[i])
    plt.plot(y_p_cnn[:,2,i]*z_p_cnn[:,i], label=appliance_labels[app], color=colors[i], linestyle="--")
plt.ylabel("Power $W$")
plt.legend()
savefig("cnn_prediction", format=".pdf")

In [187]:
appliance_labels={'kettle':"KT", 'fridge':"FRZ", 'dishwasher':"DW", 'washingmachine':"WM", 'microwave':"MW"}
fig = plt.figure(figsize=(6, 4))
for i, app in enumerate(list(appliance_data.keys())):
    plt.plot(y_t_cnn[:,i]*z_t_cnn[:,i], label=appliance_labels[app], color=colors[i])
    #plt.plot(y_p_cnn[:,2,i]t label=appliance_labels[app], color=colors[i], linestyle="--")
plt.ylabel("Power $W$")
plt.legend()
savefig("ground_truth", format=".pdf")